In [ ]:
# Step 1: Install necessary libraries
!pip install datasets evaluate transformers[sentencepiece] -q
!pip install accelerate seqeval
# To run the training on TPU, you will need to uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 92.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.

In [ ]:
!pip install -U neptune -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 3.3 MB/s eta 0:00:00


In [ ]:
huggingface_model_name = "rowdy-store/products-ner" #@param {type:"string"}
train_size = 0.8 #@param {type:"number"}
valid_size = 0.1 #@param {type:"number"}
neptune_project="alexfiorenza/Rowdy" #@param {type:"string"}
numb_labels=13

In [ ]:
from transformers.integrations import NeptuneCallback

In [ ]:
import os
from getpass import getpass

os.environ["NEPTUNE_API_TOKEN"] = getpass("Enter your Neptune API token: ")
os.environ["NEPTUNE_PROJECT"] = neptune_project

Enter your Neptune API token: ··········


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from google.colab import files

uploaded = files.upload()

# Assuming you've uploaded only one file, we'll capture the filename
file_name = list(uploaded.keys())[0]
print(f"Uploaded file: {file_name}")

Saving project-2-at-2023-11-01-23-22-edf2222e.conll to project-2-at-2023-11-01-23-22-edf2222e.conll
Uploaded file: project-2-at-2023-11-01-23-22-edf2222e.conll


In [ ]:
def read_conll(file_path):
    with open(file_path, "r") as f:
        data = f.read().strip().split("\n\n")
    return data

def red_lines(file_path):
    with open(file_path, "r") as f:
        lines = f.readlines()
    return lines

data = read_conll(file_name)
lines=red_lines(file_name)

# Split the data into train, test, and valid sets
train_size = int(train_size * len(data))
valid_size = int(valid_size * len(data))

train_data = data[:train_size]
valid_data = data[train_size: train_size + valid_size]
test_data = data[train_size + valid_size:]

with open("train.txt", "w") as f:
    f.write("\n\n".join(train_data))

with open("test.txt", "w") as f:
    f.write("\n\n".join(test_data))

with open("valid.txt", "w") as f:
    f.write("\n\n".join(valid_data))

print("Data split and written to train.txt, test.txt, and valid.txt")


ner_tags = [line.split()[3] for line in lines if len(line.strip()) > 0 and not line.startswith("-DOCSTART-")]
unique_tags = list(set(ner_tags))
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
ner_tags = [tag2id[tag] for tag in ner_tags]

Data split and written to train.txt, test.txt, and valid.txt


In [ ]:
%%writefile ner.py

import datasets

_CITATION = """\
Alex Fiorenza & Leonardo Leenen
"""

_DESCRIPTION = """\
Dataset to train a NER capable of extracting entities and perform certain actions
"""

_HOMEPAGE = "https://www.jurassiclake.com/"

_LICENSE = "MIT"

_PATHS = {
    "train": "train.txt",
    "test": "test.txt",
    "validation": "valid.txt",
}

_NER_TAGS = [
    "O",
    "B-PRODUCT_QUANTITY",
    "I-PRODUCT_QUANTITY",
    "B-PRODUCT_TYPE",
    "I-PRODUCT_TYPE",
    "B-PRODUCT_CHARACTERISTICS",
    "I-PRODUCT_CHARACTERISTICS",
    "B-PRODUCT_BRAND",
    "I-PRODUCT_BRAND",
    "B-PRODUCT_GENDER",
    "I-PRODUCT_GENDER",
    "B-PRODUCT_SIZE",
    "I-PRODUCT_SIZE"
]

NER_TAGS_DICT = {tag: id for id, tag in enumerate(_NER_TAGS)}

class NerConfig(datasets.BuilderConfig):
    """BuilderConfig for NerConfig"""

    def __init__(self, **kwargs):
        """BuilderConfig for NerConfig.
        Args:
        **kwargs: keyword arguments forwarded to super.
        """
        super(NerConfig, self).__init__(**kwargs)


class Ner(datasets.GeneratorBasedBuilder):
    """Ner dataset"""

    BUILDER_CONFIGS = [
        NerConfig(name="ner", version=datasets.Version("1.0.0"), description="Ner dataset"),
    ]

    def _info(self):
        return datasets.DatasetInfo(
            description=_DESCRIPTION,
            features=datasets.Features(
                {
                    "id": datasets.Value("string"),
                    "tokens": datasets.Sequence(datasets.Value("string")),
                    "ner_tags": datasets.Sequence(datasets.ClassLabel(names=_NER_TAGS)),
                }
            ),
            supervised_keys=None,
            homepage=_HOMEPAGE,
            license=_LICENSE,
            citation=_CITATION,
        )

    def _split_generators(self, dl_manager):
        """Returns SplitGenerators."""
        return [
            datasets.SplitGenerator(name=datasets.Split.TRAIN, gen_kwargs={"filepath": _PATHS["train"]}),
            datasets.SplitGenerator(name=datasets.Split.TEST, gen_kwargs={"filepath": _PATHS["test"]}),
            datasets.SplitGenerator(name=datasets.Split.VALIDATION, gen_kwargs={"filepath": _PATHS["validation"]}),
        ]

    def _generate_examples(self, filepath):
        """Generate examples"""
        with open(filepath, encoding="utf-8") as f:
            guid = 0
            tokens = []
            ner_tags = []
            for line in f:
                if line.startswith("-DOCSTART-") or line == "" or line == "\n":
                    if tokens:
                        yield guid, {
                            "id": str(guid),
                            "tokens": tokens,
                            "ner_tags": [NER_TAGS_DICT.get(tag, NER_TAGS_DICT["O"]) for tag in ner_tags],  # If tag is not found, map to O
                        }
                        guid += 1
                        tokens = []
                        ner_tags = []
                else:
                    splits = line.split(" ")
                    tokens.append(splits[0])
                    ner_tags.append(splits[3].rstrip())  # Adjusted index here
            # Last example
            yield guid, {
                "id": str(guid),
                "tokens": tokens,
                "ner_tags": [NER_TAGS_DICT.get(tag, NER_TAGS_DICT["O"]) for tag in ner_tags],  # If tag is not found, map to O
            }


Writing ner.py


In [ ]:
from datasets import load_dataset

raw_dataset = load_dataset('./ner.py')
print(raw_dataset)


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 475
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 60
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 59
    })
})


In [ ]:
raw_dataset["train"][0]["tokens"]

['Are',
 'there',
 'digital',
 'watches',
 'with',
 'alarm',
 'features',
 'in',
 'stock',
 '?']

In [ ]:
ner_feature = raw_dataset["train"].features["ner_tags"]
ner_feature

Sequence(feature=ClassLabel(names=['O', 'B-PRODUCT_QUANTITY', 'I-PRODUCT_QUANTITY', 'B-PRODUCT_TYPE', 'I-PRODUCT_TYPE', 'B-PRODUCT_CHARACTERISTICS', 'I-PRODUCT_CHARACTERISTICS', 'B-PRODUCT_BRAND', 'I-PRODUCT_BRAND', 'B-PRODUCT_GENDER', 'I-PRODUCT_GENDER', 'B-PRODUCT_SIZE', 'I-PRODUCT_SIZE'], id=None), length=-1, id=None)

In [ ]:
label_list = raw_dataset["train"].features[f"ner_tags"].feature.names
label_list

['O',
 'B-PRODUCT_QUANTITY',
 'I-PRODUCT_QUANTITY',
 'B-PRODUCT_TYPE',
 'I-PRODUCT_TYPE',
 'B-PRODUCT_CHARACTERISTICS',
 'I-PRODUCT_CHARACTERISTICS',
 'B-PRODUCT_BRAND',
 'I-PRODUCT_BRAND',
 'B-PRODUCT_GENDER',
 'I-PRODUCT_GENDER',
 'B-PRODUCT_SIZE',
 'I-PRODUCT_SIZE']

In [ ]:
words = raw_dataset["train"][1]["tokens"]
labels = raw_dataset["train"][1]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_list[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

How much for these Oakley          sports                    glasses        ? 
O   O    O   O     B-PRODUCT_BRAND B-PRODUCT_CHARACTERISTICS B-PRODUCT_TYPE O 


In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
tokenizer.is_fast

True

In [ ]:
inputs = tokenizer(raw_dataset["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 'are',
 'there',
 'digital',
 'watches',
 'with',
 'alarm',
 'features',
 'in',
 'stock',
 '?',
 '[SEP]']

In [ ]:
inputs.word_ids()

[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, None]

In [ ]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [ ]:
labels = raw_dataset["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[0, 0, 3, 4, 0, 5, 6, 0, 0, 0]
[-100, 0, 0, 3, 4, 0, 5, 6, 0, 0, 0, -100]


# Truncate sequences to be no longer than DistilBERT’s maximum input length:

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
# apply the preprocessing function over the entire dataset

tokenized_datasets = raw_dataset.map(
    tokenize_and_align_labels,
    batched=True,
)

Map:   0%|          | 0/475 [00:00<?, ? examples/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

Map:   0%|          | 0/59 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
## Metrics
import evaluate

metric = evaluate.load("seqeval")

In [ ]:
import numpy as np
example=raw_dataset["train"][0]
labels = [label_list[i] for i in example[f"ner_tags"]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
id2label = {i: label for i, label in enumerate(label_list)}
label2id = {v: k for k, v in id2label.items()}

In [ ]:
## Mapping ids -> labels
id2label
label2id

{'O': 0,
 'B-PRODUCT_QUANTITY': 1,
 'I-PRODUCT_QUANTITY': 2,
 'B-PRODUCT_TYPE': 3,
 'I-PRODUCT_TYPE': 4,
 'B-PRODUCT_CHARACTERISTICS': 5,
 'I-PRODUCT_CHARACTERISTICS': 6,
 'B-PRODUCT_BRAND': 7,
 'I-PRODUCT_BRAND': 8,
 'B-PRODUCT_GENDER': 9,
 'I-PRODUCT_GENDER': 10,
 'B-PRODUCT_SIZE': 11,
 'I-PRODUCT_SIZE': 12}

In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=numb_labels,
    id2label=id2label,
    label2id=label2id,
)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir=huggingface_model_name,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=8,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/integrations/integration_utils.py:1232: NeptuneWarning: The following monitoring options are disabled by default in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', and 'capture_hardware_metrics'. To enable them, set each parameter to 'True' when initializing the run. The monitoring will continue until you call run.stop() or the kernel stops. Also note: Your source files can only be tracked if you pass the path(s) to the 'source_code' argument. For help, see the Neptune docs: https://docs.neptune.ai/logging/source_code/
  self._run = init_run(**self._init_run_kwargs, **additional_neptune_kwargs)


https://app.neptune.ai/alexfiorenza/Rowdy/e/ROW-1


You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.739545,0.342105,0.373563,0.357143,0.789720
2,No log,0.403582,0.584158,0.678161,0.627660,0.886293
3,No log,0.271556,0.710526,0.775862,0.741758,0.917445
4,No log,0.228646,0.743316,0.798851,0.770083,0.931464
5,No log,0.209336,0.775956,0.816092,0.795518,0.937695
6,No log,0.189005,0.779570,0.833333,0.805556,0.945483
7,No log,0.177158,0.819672,0.862069,0.840336,0.953271
8,No log,0.174670,0.818681,0.856322,0.837079,0.953271


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 6 operations to synchronize with Neptune. Do not kill this process.
All 6 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/alexfiorenza/Rowdy/e/ROW-1/metadata


TrainOutput(global_step=240, training_loss=0.4284327189127604, metrics={'train_runtime': 671.2442, 'train_samples_per_second': 5.661, 'train_steps_per_second': 0.358, 'total_flos': 21542141061180.0, 'train_loss': 0.4284327189127604, 'epoch': 8.0})

In [ ]:
trainer.push_to_hub()

'https://huggingface.co/alexFiorenza/products-ner/tree/main/'